In [2]:
from fastapi import FastAPI
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_ollama import ChatOllama
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
path = r"C:\Users\shahe\OneDrive\Desktop\NLP Sessions\Project\Diagnosify\04012024HR8220R.pdf"
model_name = "meditron:latest"

In [4]:
loader = PyMuPDFLoader(path)

ValueError: File path C:\Users\shahe\OneDrive\Desktop\NLP Sessions\Project\Diagnosify\04012024HR8220R.pdf is not a valid file or url

In [4]:
pages = loader.load_and_split()

with open('test_file.txt', 'w') as f:
    f.write(pages[0].page_content)

In [5]:
import anthropic

client = anthropic.Anthropic(
    api_key=os.getenv("CLAUDE_API"),
)

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude"}
    ]
)
print(message.content)

ModuleNotFoundError: No module named 'anthropic'

In [18]:
llm = ChatOllama(
    model_name=model_name,
    tempreture=0
)

In [26]:
messages = [
    (
        "system",
        "You are a helpful AI doctor, who can help diagnose medical conditions and give them recommendations for treating the disease. If you don't know the answer to a specific medical inquiry, advise seeking professional help.",
    ),
    (
        "human", "I am a 20 year old male, I have been having pain in my lower left and right side of my body. I have also been experiencing nausea and vomiting. What could be the cause of this?"
    ),
]

ai_msg = llm.invoke(messages)

In [27]:
print(ai_msg.content)


Thank you for reaching out to me! I'm here to help you. The symptoms you've described are quite non-specific, but there are several possible causes that could be contributing to your discomfort. As a responsible AI doctor, I must advise you to consult with a qualified medical professional for an accurate diagnosis and appropriate treatment.

However, based on the information you've provided, here are some potential causes of your symptoms:

1. Gastrointestinal issues: The pain in your lower left and right side could be related to a gastrointestinal issue such as irritable bowel syndrome (IBS), inflammatory bowel disease (IBD), or a food poisoning. The nausea and vomiting may also be due to a gastrointestinal problem.
2. Appendicitis: Although rare, it's possible that you might have appendicitis, which can cause pain in the lower right abdomen. If left untreated, it can lead to serious complications.
3. Kidney stones or infections: Kidney stones or urinary tract infections can cause fl